# ***Feature selection and model training***

This notebook covers the process of feature selection, model training, cross-validation, evaluation, and saving of machine learning models using the merged World Happiness Report dataset. The workflow demonstrates how to preprocess the data, train and compare several regression models, select the best one, and persist the trained pipeline for later use.

Based on the previous analysis, the following features were selected for ML model training:


***Base features***
- **gdp_per_capita**
- **social_support**
- **healthy_life_expectancy**
- **freedom**

***Optional features***
- **region**
- **country**

Country and region variables were evaluated separately during training to determine the better predictor.

Imports all necessary libraries for data manipulation (`pandas`, `numpy`, `os`), machine learning (`scikit-learn`), and model persistence (`joblib`).

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

The working directory is changed so that the system can locate the dataset.

In [2]:
os.chdir("../")
print(os.getcwd())

/home/v4lentin4/etl_workshop003


In [3]:
df_model = pd.read_csv('data/merged_data.csv')

#### ***Model Definition***

- Defines three regression models: `LinearRegression`, `RidgeCV`, and `RandomForestRegressor`.
- Sets up a 5-fold cross-validation strategy.

In [4]:
models = {
    "LinearRegression": LinearRegression(),
    "RidgeCV": RidgeCV(alphas=[0.1, 1.0, 10.0]),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42)
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

### ***Training models using `country`***

#### **Feature Selection and Splitting**

- Drops columns not used for modeling (`happiness_score`, `region`, `perceptions_of_corruption`, `generosity`, `year`).
- Sets the target variable (`happiness_score`).
- Splits the data into training and test sets.

In [5]:
X = df_model.drop(['happiness_score', 'region', 'perceptions_of_corruption', 'generosity', 'year'], axis=1)
y = df_model['happiness_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### ***Data Preprocessing***

- Identifies numeric (`gdp_per_capita`, `social_support`, `healthy_life_expectancy`, `freedom`) and categorical (`country` or `region`) features.
- Applies `StandardScaler` to numeric features and `OneHotEncoder` to categorical features using a `ColumnTransformer`.
- Transforms the training and test sets accordingly.

In [6]:
numeric_features = ['gdp_per_capita', 'social_support', 'healthy_life_expectancy', 'freedom']
categorical_features = ['country']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

#### ***Model Evaluation via Cross-Validation***

- For each model:
  - Constructs a pipeline combining preprocessing and the regressor.
  - Runs 5-fold cross-validation on the training data.
  - Evaluates performance using R², MAE, and RMSE metrics.
  - Prints performance for each fold and summarizes average and standard deviation for each metric.
- Stores results for later comparison.

In [7]:
print("CROSS VALIDATION (Training data):\n")

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('regressor', model)
    ])

    print(f"-> Evaluating model: {name}")
    r2s, maes, rmses = [], [], []

    for i, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict(X_val)

        r2 = r2_score(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        r2s.append(r2)
        maes.append(mae)
        rmses.append(rmse)

        print(f" Fold {i+1}: R2={r2:.4f}, MAE={mae:.4f}, RMSE={rmse:.4f}")

    results[name] = {
        "R2_mean": np.mean(r2s),
        "R2_std": np.std(r2s),
        "MAE_mean": np.mean(maes),
        "MAE_std": np.std(maes),
        "RMSE_mean": np.mean(rmses),
        "RMSE_std": np.std(rmses)
    }

print("\nCROSS VALIDATION SUMMARY:")
for name, res in results.items():
    print(f"\nModel: {name}")
    print(f" R2   → Mean: {res['R2_mean']:.4f} | Deviation: {res['R2_std']:.4f}")
    print(f" MAE  → Mean: {res['MAE_mean']:.4f} | Deviation: {res['MAE_std']:.4f}")
    print(f" RMSE → Mean: {res['RMSE_mean']:.4f} | Deviation: {res['RMSE_std']:.4f}")


CROSS VALIDATION (Training data):

-> Evaluating model: LinearRegression
 Fold 1: R2=0.9163, MAE=0.2065, RMSE=0.3229
 Fold 2: R2=0.9372, MAE=0.2014, RMSE=0.2926
 Fold 3: R2=0.9390, MAE=0.1923, RMSE=0.2739
 Fold 4: R2=0.9175, MAE=0.2093, RMSE=0.3270
 Fold 5: R2=0.8903, MAE=0.2397, RMSE=0.3630
-> Evaluating model: RidgeCV
 Fold 1: R2=0.9078, MAE=0.2243, RMSE=0.3389
 Fold 2: R2=0.9340, MAE=0.2082, RMSE=0.3000
 Fold 3: R2=0.9289, MAE=0.2079, RMSE=0.2957
 Fold 4: R2=0.9487, MAE=0.1814, RMSE=0.2579
 Fold 5: R2=0.9027, MAE=0.2359, RMSE=0.3420
-> Evaluating model: RandomForest
 Fold 1: R2=0.8270, MAE=0.3518, RMSE=0.4643
 Fold 2: R2=0.8547, MAE=0.3502, RMSE=0.4450
 Fold 3: R2=0.8380, MAE=0.3478, RMSE=0.4463
 Fold 4: R2=0.8637, MAE=0.3294, RMSE=0.4203
 Fold 5: R2=0.7780, MAE=0.3919, RMSE=0.5165

CROSS VALIDATION SUMMARY:

Model: LinearRegression
 R2   → Mean: 0.9201 | Deviation: 0.0176
 MAE  → Mean: 0.2098 | Deviation: 0.0160
 RMSE → Mean: 0.3159 | Deviation: 0.0307

Model: RidgeCV
 R2   → Mean:

#### ***Model Selection***


- Compares mean R² scores and selects the best-performing model based on cross-validation results.
- Prints the chosen model.

In [8]:
best_model_name = max(results, key=lambda x: results[x]['R2_mean'])
best_model = models[best_model_name]
print(f"\n -> Best model according to cross validation: {best_model_name}")


 -> Best model according to cross validation: RidgeCV


#### ***Final Training and Evaluation***

- Re-trains the pipeline using the full training set with the best model.
- Evaluates the trained pipeline on the held-out test set, printing final R², MAE, and RMSE.

In [ ]:
final_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', best_model)
])
final_pipeline.fit(X_train, y_train)

In [11]:
y_pred_test = final_pipeline.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\n -> FINAL EVALUATION (Testing data):")
print(f" R2: {r2_test:.4f}")
print(f" MAE: {mae_test:.4f}")
print(f" RMSE: {rmse_test:.4f}")


 -> FINAL EVALUATION (Testing data):
 R2: 0.9502
 MAE: 0.1839
 RMSE: 0.2487


### ***Training models using `Region`***

The process from the previous cells is replicated here to evaluate how modifying the categorical geographical feature impacts the model's performance

In [9]:
X = df_model.drop(['happiness_score', 'country', 'perceptions_of_corruption', 'generosity', 'year'], axis=1)
y = df_model['happiness_score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
numeric_features = ['gdp_per_capita', 'social_support', 'healthy_life_expectancy', 'freedom']
categorical_features = ['region']

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)

In [11]:
models = {
    "LinearRegression": LinearRegression(),
    "RidgeCV": RidgeCV(alphas=[0.1, 1.0, 10.0]),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42)
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

In [12]:
print("CROSS VALIDATION (Training data):\n")

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessing', preprocessor),
        ('regressor', model)
    ])

    print(f"-> Evaluating model: {name}")
    r2s, maes, rmses = [], [], []

    for i, (train_idx, val_idx) in enumerate(cv.split(X_train)):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict(X_val)

        r2 = r2_score(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        r2s.append(r2)
        maes.append(mae)
        rmses.append(rmse)

        print(f" Fold {i+1}: R2={r2:.4f}, MAE={mae:.4f}, RMSE={rmse:.4f}")

    results[name] = {
        "R2_mean": np.mean(r2s),
        "R2_std": np.std(r2s),
        "MAE_mean": np.mean(maes),
        "MAE_std": np.std(maes),
        "RMSE_mean": np.mean(rmses),
        "RMSE_std": np.std(rmses)
    }

print("\nCROSS VALIDATION SUMMARY:")
for name, res in results.items():
    print(f"\nModel: {name}")
    print(f" R2   → Mean: {res['R2_mean']:.4f} | Deviation: {res['R2_std']:.4f}")
    print(f" MAE  → Mean: {res['MAE_mean']:.4f} | Deviation: {res['MAE_std']:.4f}")
    print(f" RMSE → Mean: {res['RMSE_mean']:.4f} | Deviation: {res['RMSE_std']:.4f}")


CROSS VALIDATION (Training data):

-> Evaluating model: LinearRegression
 Fold 1: R2=0.7955, MAE=0.3808, RMSE=0.5047
 Fold 2: R2=0.8474, MAE=0.3527, RMSE=0.4561
 Fold 3: R2=0.7975, MAE=0.4077, RMSE=0.4990
 Fold 4: R2=0.8419, MAE=0.3477, RMSE=0.4527
 Fold 5: R2=0.7380, MAE=0.4380, RMSE=0.5612
-> Evaluating model: RidgeCV
 Fold 1: R2=0.7954, MAE=0.3807, RMSE=0.5049
 Fold 2: R2=0.8475, MAE=0.3525, RMSE=0.4559
 Fold 3: R2=0.7974, MAE=0.4077, RMSE=0.4991
 Fold 4: R2=0.8420, MAE=0.3475, RMSE=0.4526
 Fold 5: R2=0.7380, MAE=0.4380, RMSE=0.5611
-> Evaluating model: RandomForest
 Fold 1: R2=0.8529, MAE=0.3335, RMSE=0.4281
 Fold 2: R2=0.8510, MAE=0.3439, RMSE=0.4505
 Fold 3: R2=0.8422, MAE=0.3500, RMSE=0.4405
 Fold 4: R2=0.8362, MAE=0.3434, RMSE=0.4607
 Fold 5: R2=0.7861, MAE=0.3931, RMSE=0.5071

CROSS VALIDATION SUMMARY:

Model: LinearRegression
 R2   → Mean: 0.8040 | Deviation: 0.0395
 MAE  → Mean: 0.3854 | Deviation: 0.0340
 RMSE → Mean: 0.4947 | Deviation: 0.0395

Model: RidgeCV
 R2   → Mean:

In [13]:
best_model_name = max(results, key=lambda x: results[x]['R2_mean'])
best_model = models[best_model_name]
print(f"\n-> Best model according to cross validation: {best_model_name}")


-> Best model according to cross validation: RandomForest


In [14]:
final_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('regressor', best_model)
])
final_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['gdp_per_capita',
                                                   'social_support',
                                                   'healthy_life_expectancy',
                                                   'freedom']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['region'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [15]:
y_pred_test = final_pipeline.predict(X_test)
r2_test = r2_score(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

print(f"\n -> FINAL EVALUATION (Testing data):")
print(f" R2: {r2_test:.4f}")
print(f" MAE: {mae_test:.4f}")
print(f" RMSE: {rmse_test:.4f}")


 -> FINAL EVALUATION (Testing data):
 R2: 0.8227
 MAE: 0.3517
 RMSE: 0.4693


Compared to the country-based setup, this configuration shows significantly lower performance. The R² on test data drops from 0.9542 to 0.8227, and both MAE and RMSE increase considerably.

### ***Model Persistence***
- Saves the final trained pipeline (including preprocessing and the best regressor) as a `.pkl` file for later use (`model/trained_model.pkl`).

In [ ]:
joblib.dump(final_pipeline, 'model/trained_model.pkl')
print(f"\n->  Model '{best_model_name}' trained and saved as 'trained_model.pkl'")


->  Model 'RandomForest' trained and saved as 'trained_model.pkl'


Although the models trained with country show better performance and don't seem to have overfitting, there remains concern that they may not generalize well when new data arrives, therefore we opted to use the model with the best performance trained using region, which in this case was RandomForest with $R^2 = 0.8337$.